# Organizacion del NoteBook
Dado un review, el bot deicidirá si tiene un review positivo o negativo. Específicamente:

Primero se extraen características para la regresión logística texto de entrenamiento
Se implementa la regresión logística
Se aplica regresión logística en una tarea de procesamiento de lenguaje natural
Prueba usando tu regresión logística
Realizar análisis de errores
Se ejecuta un conjunto de datos de reseñas de peliculas.
Ejecute la celda a continuación para cargar los paquetes.


In [5]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
import matplotlib.pyplot as plt
import gensim
import spacy
nltk.download('stopwords')
nltk.download('punkt')
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import json

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


El archivo usado se encuentra en la ruta Recommender Systems/recsys/Data. En la siguiente celda se muestra un ejemplo de lo que contiene.

In [6]:
df_news = pd.read_csv("Data/training_data.csv")
df_news.drop('Unnamed: 0', inplace=True, axis=1)

df_news.head(2)
df_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 812 entries, 0 to 811
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  812 non-null    object
 1   score   812 non-null    object
dtypes: object(2)
memory usage: 12.8+ KB


### Preprocesamiento de la Data:
Para poder aprovechar mejor la data, primero se limpiarán los textos de tal forma que solo se evalue la información relevante que contienen.
Por tal motivo, se buscan eliminar palabras que no son necesarias como por ejemplo aquellas que esten en otros idiomas, palabras innecesarias y signos de puntuación. Utilizaremos el modulo stopwords de NLTK para este proceso. 

In [7]:
black_list = ['más', 'mas', 'unir', 'paises', 'pais', 'espa', 'no', 'os', 'a', 'compa', 'acompa', 'off', 'and', 'grecia', 'the','it', 'to',
              'd',  'et',  'dame',  'il',  'dans', 'that',  'as',   'for',  'it',  'elections',  'would',  'this',  'with', 'york', 'obama', 'chavez', 'gadafi']
stop = set(stopwords.words('spanish'))
additional_stopwords=set(black_list)
stopwords = stop.union(additional_stopwords)
nlp = spacy.load('es_core_news_sm')

Adicionalmente creamos una serie de funciones para limpiar los textos de cada review, y solo quedarnos con las raices de las palabras en la mayoria de los casos, y eliminar ruido de palabras inncesarias.
La funcio cleaner primero utiliza expresiones regulares, para eliminar tildes, urls y símbolos. Luego lematizaremos, es decir, convertimos ciertas palabras que funcionan mejor cuando estan juntas y solo dejamos adjetivos y sustantivos, luego stemmizamos, es decir dejamos la raiz de una palabra. 

In [8]:
def cleaner(word):
  """Build clean text.
    Input:
        word: a string of tweets

    Output:
        out_text: a list with lemmatize and stemmed and eliminated unnecesary words

  """
  word = re.sub(r'((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*', '', word, flags=re.MULTILINE)
  word = re.sub(r'(?::|;|=)(?:-)?(?:\)|\(|D|P)', "", word)
  word = re.sub(r'ee.uu', 'eeuu', word)
  word = re.sub(r'\#\.', '', word)
  word = re.sub(r'\n', '', word)
  word = re.sub(r',', '', word)
  word = re.sub(r'\-', ' ', word)
  word = re.sub(r'\.{3}', ' ', word)
  word = re.sub(r'a{2,}', 'a', word)
  word = re.sub(r'é{2,}', 'é', word)
  word = re.sub(r'i{2,}', 'i', word)
  word = re.sub(r'ja{2,}', 'ja', word) 
  word = re.sub(r'á', 'a', word)
  word = re.sub(r'é', 'e', word)
  word = re.sub(r'í', 'i', word)
  word = re.sub(r'ó', 'o', word)
  word = re.sub(r'ú', 'u', word)  
  word = re.sub('[^a-zA-Z]', ' ', word)
  list_word_clean = []
  for w1 in word.split(" "):
    if  w1.lower() not in stopwords:
      list_word_clean.append(w1.lower())

  bigram_list = bigram[list_word_clean]  # use of bigram anda lemmatization to check if there are word that work better together
  out_text = lemmatization(" ".join(bigram_list))
  stemmer = SnowballStemmer('spanish') # use of stemmer to eliminate suffix in words, NLTK recommends SnowBall but, it can be used other stemmers.
  out_text = stemming(out_text, stemmer)
  return out_text

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ']):
    """Lemmatize text.
    Input:
        texts: a list with the words of a text
        allowed_postags: a list with Part of Speech used by spacy. Check out https://spacy.io/usage/linguistic-features for more options.
    Output:
        out_text: a list with lemmatize 

    """
    texts_out = [ token.text for token in nlp(texts) if token.pos_ in 
                 allowed_postags and token.text not in black_list and len(token.text)>2]
    return texts_out

def stemming(text_list, stemmer):
    """Lemmatize text.
    Input:
        texts: a list with the words of a text
        stemmer: Stemmer used by NLTK module. Check out https://www.nltk.org/api/nltk.stem.html#module-nltk.stem for more options.
    Output:
        out_text: a list with stemmed text

    """
    review_clean = []
    for word in text_list:
        stem_word = stemmer.stem(word)  # stemming word
        review_clean.append(stem_word)

    return review_clean

def build_freqs(tweets, ys):
    """Build frequencies.
    Input:
        tweets: a list of tweets
        ys: an m x 1 array with the sentiment label of each tweet
            (either 0 or 1)
    Output:
        freqs: a dictionary mapping each (word, sentiment) pair to its
        frequency
    """
    # Convert np array to list since zip needs an iterable.
    # The squeeze is necessary or the list ends up with one element.
    # Also note that this is just a NOP if ys is already a list.
    yslist = np.squeeze(ys).tolist()

    # Start with an empty dictionary and populate it by looping over all tweets
    # and over all processed words in each tweet.
    freqs = {}
    for y, tweet in zip(yslist, tweets):
        
        for word in tweet[0]:
            pair = (word, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1

    return freqs

def normalize_bad_good_review(score):
    """Build numerical score.
    Input:
        score: a text showing if a movie is good or bad
      
    Output:
        num_score: a numerical represetation for a score
        
    """
    if score == 'buena':
        return 1.0
    else:
        return 0.0

#### Ejemplo de una review luego de ser preprocesada:
Aplicamos la funcion creada en todo el Dataframe y creamos una columna que contenga el resultado.
Se hace lo mismo para el score.

In [9]:
bigram = gensim.models.Phrases(df_news.review.to_list())
df_news.review[0]
cleaner(df_news.review[0])

['necesari',
 'coraj',
 'acontec',
 'recient',
 'traumat',
 'atent',
 'septiembr',
 'greengrass',
 'adem',
 'regal',
 'peliculon',
 'sab',
 'sucedi',
 'septiembr',
 'sig',
 'detall',
 'revel',
 'necesitari',
 'avion',
 'dich',
 'sufri',
 'ataqu',
 'torr',
 'gemel',
 'estrell',
 'pennsylvani',
 'necesari',
 'respet',
 'victim',
 'famili',
 'condicion',
 'indispens',
 'pelicul',
 'guion',
 'direccion',
 'greengrass',
 'ejemplar',
 'adem',
 'incurr',
 'estereotip',
 'pens',
 'respect',
 'figur',
 'terror',
 'retrat',
 'objet',
 'crim',
 'brutal',
 'excus',
 'posibl',
 'hech',
 'gent',
 'normal',
 'semej',
 'atroc',
 'terribl',
 'car',
 'mal',
 'cicatriz',
 'rostr',
 'fin',
 'real',
 'creibl',
 'histori',
 'estil',
 'cin',
 'documental',
 'ayud',
 'pelicul',
 'utilizacion',
 'actor',
 'desconoc',
 'autent',
 'control',
 'escen',
 'pes',
 'cint',
 'ficcion',
 'fidel',
 'hech',
 'deberi',
 'opinion',
 'exactitud',
 'sucedi',
 'pelicul',
 'divid',
 'part',
 'diferenci',
 'primer',
 'gestacion

In [10]:
df_news['clean_review'] = df_news['review'].apply(cleaner)

In [11]:
df_news.head()

,review,score,clean_review
0,Era necesario mucho coraje para abordar aconte...,buena,"[necesari, coraj, acontec, recient, traumat, a..."
1,Esperaba con curiosidad y ciertas ganas el est...,mala,"[curi, gan, estren, nuev, pelicul, antoni, ban..."
2,"Wes Craven, convertido en factoría, nos vuelve...",mala,"[wes, convert, factori, prescind, histori, rei..."
3,Va la gente y se rasga las vestiduras con 'Caó...,mala,"[gent, rasg, vestidur, caotic, nuev, pelicul, ..."
4,Director: Mariano Ozores.Duración: 77 minutos....,buena,"[director, marian, diciembr, ser, mansion, sub..."


In [12]:
df_news['normalize_score'] = df_news['score'].apply(normalize_bad_good_review)

In [13]:
df_news.head(20)


,review,score,clean_review,normalize_score
0,Era necesario mucho coraje para abordar aconte...,buena,"[necesari, coraj, acontec, recient, traumat, a...",1.0
1,Esperaba con curiosidad y ciertas ganas el est...,mala,"[curi, gan, estren, nuev, pelicul, antoni, ban...",0.0
2,"Wes Craven, convertido en factoría, nos vuelve...",mala,"[wes, convert, factori, prescind, histori, rei...",0.0
3,Va la gente y se rasga las vestiduras con 'Caó...,mala,"[gent, rasg, vestidur, caotic, nuev, pelicul, ...",0.0
4,Director: Mariano Ozores.Duración: 77 minutos....,buena,"[director, marian, diciembr, ser, mansion, sub...",1.0
5,Si por algo tendremos que recordar Serpientes ...,mala,"[serpient, avion, futur, ser, trastiend, food,...",0.0
6,El ex Monthy Python Terry Gilliam ha sabido cr...,mala,"[monthy, gilliam, sab, carrer, fam, john, camp...",0.0
7,"Existen cierto tipo de películas dirigidas, en...",mala,"[tip, pelicul, dirig, mayori, adolescent, unic...",0.0
8,Qué difícil es encontrar una película cuya amb...,mala,"[dificil, pelicul, ambientacion, clim, intach,...",0.0
9,La negrura tremebunda de la España del sabañón...,buena,"[negrur, tremebund, caf, hambr, pisit, luz, re...",1.0


### Procesamiento de la Data y entrenamiento del algoritmo:
Luego de limpiar la data procedemos a procesarla, es decir convertir el texto limpio en informacion que pueda ser evaluada, es decir copnvertir en numeros, y preferiblemente en vectores que representen la información. Existen varios métodos, espacio de vectores y producto interno, PCA, Word2Vec, Glove, para este ejercicio usaremos frecuencia de apariciones de palabras en textos. Los pasos a seguir serían:
* Convertir listas en vectores usando numpy
* Extracción de caracteristicas de reviews
* Entrenamiento de algoritmo 
* Testeo de algoritmo

Conversion de los textos y scores en listas, adicional a eso partiremos el set que se ha recibido en dos: Uno para el entrenamiento del algoritmo y otro para el testeo y medir la precision. Para el split utilizamos sklearn cuyo modulo tiene una funcion de particion de datasets.

In [14]:
x_array = np.asarray(df_news['clean_review'])
y_array = np.asarray(df_news['normalize_score'])

In [15]:
x_array

array([list(['necesari', 'coraj', 'acontec', 'recient', 'traumat', 'atent', 'septiembr', 'greengrass', 'adem', 'regal', 'peliculon', 'sab', 'sucedi', 'septiembr', 'sig', 'detall', 'revel', 'necesitari', 'avion', 'dich', 'sufri', 'ataqu', 'torr', 'gemel', 'estrell', 'pennsylvani', 'necesari', 'respet', 'victim', 'famili', 'condicion', 'indispens', 'pelicul', 'guion', 'direccion', 'greengrass', 'ejemplar', 'adem', 'incurr', 'estereotip', 'pens', 'respect', 'figur', 'terror', 'retrat', 'objet', 'crim', 'brutal', 'excus', 'posibl', 'hech', 'gent', 'normal', 'semej', 'atroc', 'terribl', 'car', 'mal', 'cicatriz', 'rostr', 'fin', 'real', 'creibl', 'histori', 'estil', 'cin', 'documental', 'ayud', 'pelicul', 'utilizacion', 'actor', 'desconoc', 'autent', 'control', 'escen', 'pes', 'cint', 'ficcion', 'fidel', 'hech', 'deberi', 'opinion', 'exactitud', 'sucedi', 'pelicul', 'divid', 'part', 'diferenci', 'primer', 'gestacion', 'tragedi', 'terror', 'viaj', 'aeropuert', 'prepar', 'tripulacion', 'avion'

In [16]:
y_array

array([1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 1.,
       1., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1.,
       1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1.,
       0., 1., 1., 1., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 0., 0.,
       0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 1.,
       1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 0., 1., 1.,
       1., 1., 0., 0., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1.,
       1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1.,
       1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0.,
       1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0.,
       0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 0., 1., 1., 1.,
       0., 0., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 0., 1., 1.,
       1., 1., 1., 1., 1.

In [17]:
x_train, x_test, y_train, y_test = train_test_split(x_array, y_array)

In [18]:
x_train = x_train[:,np.newaxis]
x_test =  x_test[:,np.newaxis]
y_train = y_train[:,np.newaxis]
y_test = y_test[:,np.newaxis]

Como resultado, obtenemos 4 vectores: 2 de entrenamiento y 2 de test. Con esta información, utilizamos los vectores de entrenamiento para extraer las caracteríticas de cada review, que es vbasicamente asignar la frecuencia de aparición a cada palabra en un review bueno o malo. enteoría, las palabras que más aparezcan en un texto bueno ayudarán a identificar si el review es bueno y lo mismo para una review mala.

In [19]:
x_train.shape

(609, 1)

In [20]:
y_train.shape

(609, 1)

In [21]:
freqs = build_freqs(x_train, y_train)
freqs

{('gan', 0.0): 33,
 ('nuev', 0.0): 132,
 ('propuest', 0.0): 18,
 ('eli', 0.0): 9,
 ('roth', 0.0): 12,
 ('inerci', 0.0): 2,
 ('sensacion', 0.0): 17,
 ('previ', 0.0): 3,
 ('pelicul', 0.0): 574,
 ('mal', 0.0): 191,
 ('esquem', 0.0): 7,
 ('simil', 0.0): 6,
 ('anterior', 0.0): 42,
 ('vez', 0.0): 146,
 ('chic', 0.0): 66,
 ('femin', 0.0): 2,
 ('mied', 0.0): 27,
 ('bell', 0.0): 11,
 ('muj', 0.0): 35,
 ('balneari', 0.0): 2,
 ('eslovaqui', 0.0): 1,
 ('innovacion', 0.0): 3,
 ('secuel', 0.0): 35,
 ('lad', 0.0): 61,
 ('tortur', 0.0): 12,
 ('ric', 0.0): 10,
 ('american', 0.0): 56,
 ('primer', 0.0): 110,
 ('fil', 0.0): 8,
 ('caceri', 0.0): 2,
 ('part', 0.0): 143,
 ('ahi', 0.0): 36,
 ('topic', 0.0): 39,
 ('envuelt', 0.0): 4,
 ('guion', 0.0): 171,
 ('entiend', 0.0): 1,
 ('dej', 0.0): 16,
 ('golp', 0.0): 10,
 ('humor', 0.0): 36,
 ('negr', 0.0): 37,
 ('lug', 0.0): 49,
 ('final', 0.0): 148,
 ('protagoniz', 0.0): 13,
 ('diabl', 0.0): 6,
 ('hostel', 0.0): 16,
 ('chicl', 0.0): 1,
 ('curios', 0.0): 22,
 ('pro

 Luego de obtener las frecuencias asignadas para cada tupla de palabra y score, procederemos a extraer las características, que consiste en convertir todas las palabras en un vector representativo del texto. Para el ejercicio el vector resultante es de 1 x 3.
 Como algoritmo de clasifición se usa como kernel la regresión logística, cuya funcion viene dada por:
 $$ h(z) = \frac{1}{1+\exp^{-z}} \tag{1}$$
Donde $z$ corresponde a:

$$z = \theta_0 x_0 + \theta_1 x_1 + \theta_2 x_2$$
con $[x_0,x_1,x_2]$ el vector representativo  del review y $[\theta_0, \theta_1, \theta_2]$ los pesos optimos para identificar si una review es buena o mala. Para obtener estos pesos se utiliza el algoritmo de gradiente descendet en este ejercicio, que viene dado por:
$$\mathbf{\theta} = \mathbf{\theta} - \frac{\alpha}{m} \times \left( \mathbf{x}^T \cdot \left( \mathbf{h-y} \right) \right)$$
 

In [22]:
def extract_features(tweet, freqs):
    '''
    Input: 
        tweet: a list of words for one tweet
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
    Output: 
        x: a feature vector of dimension (1,3)
    '''
    # process_tweet tokenizes, stems, and removes stopwords
    word_l = tweet
    
    # 3 elements in the form of a 1 x 3 vector
    x = np.zeros((1, 3)) 
    
    #bias term is set to 1
    x[0,0] = 1 
    
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    
    # loop through each word in the list of words
    for word in word_l:
        
        # increment the word count for the positive label 1
        x[0,1] += freqs.get((word, 1.0),0)
        
        # increment the word count for the negative label 0
        x[0,2] += freqs.get((word, 0.0),0)
        
    ### END CODE HERE ###
    assert(x.shape == (1, 3))
    return x

def sigmoid(z): 
    '''
    Input:
        z: is the input (can be a scalar or an array)
    Output:
        h: the sigmoid of z
    '''
    
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    # calculate the sigmoid of z
    h = 1 / (1 + np.exp(-z))
    ### END CODE HERE ###
    
    return h

# UNQ_C2 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def gradientDescent(x, y, theta, alpha, num_iters):
    '''
    Input:
        x: matrix of features which is (m,n+1)
        y: corresponding labels of the input matrix x, dimensions (m,1)
        theta: weight vector of dimension (n+1,1)
        alpha: learning rate
        num_iters: number of iterations you want to train your model for
    Output:
        J: the final cost
        theta: your final weight vector
    Hint: you might want to print the cost to make sure that it is going down.
    '''
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    # get 'm', the number of rows in matrix x
    m = x.shape[0]
    
    for i in range(0, num_iters):
        
        # get z, the dot product of x and theta
        z = np.dot(x,theta)
        
        # get the sigmoid of z
        h = sigmoid(z)
        
        # calculate the cost function
        J = -1./m * (np.dot(y.transpose(), np.log(h)) + np.dot((1-y).transpose(),np.log(1-h)))    

        # update the weights theta
        theta = theta = theta - (alpha/m) * np.dot(x.transpose(),(h-y))
        
    ### END CODE HERE ###
    J = float(J)
    return J, theta




# UNQ_C5 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def test_logistic_regression(test_x, test_y, freqs, theta):
    """
    Input: 
        test_x: a list of tweets
        test_y: (m, 1) vector with the corresponding labels for the list of tweets
        freqs: a dictionary with the frequency of each pair (or tuple)
        theta: weight vector of dimension (3, 1)
    Output: 
        accuracy: (# of tweets classified correctly) / (total # of tweets)
    """
    
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    
    # the list for storing predictions
    y_hat = []
    
    for tweet in test_x:
        # get the label prediction for the tweet
        y_pred = predict_tweet(tweet[0], freqs, theta)
        
        if y_pred > 0.5:
            # append 1.0 to the list
            y_hat.append(1)
        else:
            # append 0 to the list
            y_hat.append(0)

    # With the above implementation, y_hat is a list, but test_y is (m,1) array
    # convert both to one-dimensional arrays in order to compare them using the '==' operator
    accuracy = (y_hat==np.squeeze(test_y)).sum()/len(test_x)

    ### END CODE HERE ###
    
    return accuracy

In [23]:
print(x_train[0,0])
tmp1 = extract_features(x_train[0,0], freqs)
print(tmp1)
tmp2 = extract_features('blorb bleeeeb bloooob', freqs)
print(tmp2)

['gan', 'nuev', 'propuest', 'eli', 'roth', 'inerci', 'sensacion', 'previ', 'pelicul', 'mal', 'mal', 'esquem', 'simil', 'anterior', 'vez', 'chic', 'femin', 'mied', 'bell', 'muj', 'chic', 'balneari', 'eslovaqui', 'innovacion', 'secuel', 'roth', 'lad', 'tortur', 'ric', 'american', 'primer', 'vez', 'fil', 'caceri', 'part', 'ahi', 'topic', 'topic', 'envuelt', 'guion', 'entiend', 'dej', 'golp', 'humor', 'negr', 'lug', 'final', 'protagoniz', 'diabl', 'hostel', 'chicl', 'curios', 'protagon', 'exces', 'respet', 'bikini', 'balneari', 'cambi', 'cos', 'came', 'gui', 'hostel', 'relat', 'elisabeth', 'bathory', 'actor', 'ambientacion', 'bellez', 'localiz', 'llamat', 'tortur', 'llam', 'hostel']
[[1.000e+00 4.946e+03 3.448e+03]]
[[1. 0. 0.]]


In [24]:
X = np.zeros((len(x_train), 3))
print(X.shape)
for i in range(len(x_train)):
    print(x_train[i,0])
    X[i, :]= extract_features(x_train[i,0], freqs)
    print(X[i, :])

(609, 3)
['gan', 'nuev', 'propuest', 'eli', 'roth', 'inerci', 'sensacion', 'previ', 'pelicul', 'mal', 'mal', 'esquem', 'simil', 'anterior', 'vez', 'chic', 'femin', 'mied', 'bell', 'muj', 'chic', 'balneari', 'eslovaqui', 'innovacion', 'secuel', 'roth', 'lad', 'tortur', 'ric', 'american', 'primer', 'vez', 'fil', 'caceri', 'part', 'ahi', 'topic', 'topic', 'envuelt', 'guion', 'entiend', 'dej', 'golp', 'humor', 'negr', 'lug', 'final', 'protagoniz', 'diabl', 'hostel', 'chicl', 'curios', 'protagon', 'exces', 'respet', 'bikini', 'balneari', 'cambi', 'cos', 'came', 'gui', 'hostel', 'relat', 'elisabeth', 'bathory', 'actor', 'ambientacion', 'bellez', 'localiz', 'llamat', 'tortur', 'llam', 'hostel']
[1.000e+00 4.946e+03 3.448e+03]
['joy', 'desalm', 'dur', 'diamant', 'ultim', 'rob', 'bols', 'parej', 'famili', 'feliz', 'metr', 'disfraz', 'beb', 'parej', 'fin', 'escen', 'clav', 'sabotaj', 'hitchcock', 'palaci', 'dam', 'socied', 'realid', 'espi', 'nazi', 'escen', 'nivel', 'lad', 'hero', 'novi', 'espi'

[1.0000e+00 2.0186e+04 9.9620e+03]
['larg', 'histori', 'cin', 'recurr', 'mayor', 'menor', 'fortun', 'distint', 'ramif', 'grup', 'criminal', 'organiz', 'larg', 'planet', 'hech', 'part', 'oscur', 'libr', 'atadur', 'moral', 'social', 'grup', 'organiz', 'element', 'principi', 'integr', 'grup', 'cre', 'espect', 'atraccion', 'inmoral', 'personaj', 'explot', 'opcion', 'mafi', 'italian', 'ramif', 'obra', 'cumbr', 'padrin', 'coppol', 'obvi', 'referent', 'mafi', 'irlandes', 'tri', 'chin', 'recurrent', 'cald', 'cultiv', 'cin', 'kong', 'extenuacion', 'result', 'mayori', 'cas', 'notabl', 'grup', 'motiv', 'analisis', 'certer', 'defect', 'dign', 'cas', 'puntual', 'cas', 'mafi', 'rus', 'quiz', 'deb', 'secret', 'extint', 'inculc', 'miembr', 'desinformacion', 'fronter', 'oportun', 'tratamient', 'realist', 'cercan', 'desconoc', 'proyect', 'cineast', 'curt', 'terren', 'fantast', 'dav', 'pareci', 'principi', 'aventur', 'decision', 'realid', 'director', 'canadiens', 'habi', 'demostr', 'ultim', 'proyect', 'h

[1.0000e+00 1.3802e+04 8.1280e+03]
['pelicul', 'incapac', 'terc', 'reduc', 'grup', 'film', 'repart', 'juici', 'nuremberg', 'vencedor', 'venc', 'afirmacion', 'podri', 'exager', 'principi', 'elenc', 'transform', 'logic', 'nombr', 'tracy', 'burt', 'richard', 'widmark', 'schell', 'dietrich', 'montgomery', 'clift', 'garland', 'film', 'narr', 'proces', 'jerarc', 'nazis', 'deten', 'guerr', 'realiz', 'despu', 'fin', 'guerr', 'pos', 'tranquil', 'dolor', 'ira', 'permit', 'caracteriz', 'interpret', 'magnif', 'pelicul', 'alla', 'facil', 'perdedor', 'guerr', 'barbar', 'ahond', 'personal', 'sentimient', 'antigu', 'magistr', 'actuacion', 'regim', 'interaccion', 'juez', 'inolvid', 'tracy', 'personaj', 'interpret', 'dietrich', 'product', 'enorm', 'calid', 'extens', 'metraj', 'hor', 'delicadez', 'relat', 'hech', 'firmez', 'escabr', 'dud', 'merit', 'stanley', 'kram', 'abby', 'mann', 'director', 'guionist', 'venc', 'alard', 'tecnic', 'necesari', 'part', 'pes', 'argument', 'reca', 'dialog', 'vez', 'gran', 

[1.0000e+00 1.3264e+04 6.4290e+03]
['moscow', 'zer', 'patater', 'brom', 'facil', 'pelicul', 'huev', 'fin', 'bast', 'mejor', 'pelicul', 'prens', 'termin', 'despu', 'bod', 'result', 'cas', 'moscow', 'zer', 'sali', 'fri', 'lluvios', 'cuent', 'nuev', 'punt', 'vist', 'nuev', 'enganch', 'asust', 'fin', 'verd', 'largometraj', 'director', 'ola', 'aka', 'mari', 'ambient', 'subsuel', 'padr', 'owen', 'amig', 'antropolog', 'sergei', 'karparov', 'desaparec', 'extra', 'circunst', 'leyend', 'socied', 'marginal', 'moscovit', 'busqued', 'hombr', 'par', 'mercenari', 'colabor', 'muj', 'habri', 'llev', 'ultim', 'miss', 'call', 'junt', 'decen', 'angost', 'pasill', 'derech', 'vist', 'aqui', 'misteri', 'trill', 'vaci', 'olor', 'mund', 'paralel', 'piens', 'demoni', 'leyend', 'enigmat', 'final', 'sorpres', 'fin', 'aburr', 'claustrofobi', 'cansanci', 'mied', 'motiv', 'protagon', 'clar', 'cas', 'profund', 'adem', 'personaj', 'plan', 'total', 'ausenci', 'trabaj', 'dialog', 'molest', 'monolog', 'sergei', 'karparov

['littl', 'miss', 'mejor', 'guion', 'pas', 'refier', 'cuant', 'conten', 'tambi', 'cuant', 'pelis', 'accion', 'intern', 'dificil', 'adem', 'aburr', 'buen', 'guion', 'vec', 'entreten', 'hech', 'hech', 'littl', 'miss', 'trabaj', 'guion', 'dialog', 'triangulacion', 'personaj', 'peli', 'monton', 'subtram', 'tram', 'principal', 'concurs', 'guionist', 'michael', 'arndt', 'mont', 'estructur', 'relacion', 'viaj', 'frenet', 'personaj', 'propi', 'vid', 'guion', 'propi', 'estructur', 'tram', 'conten', 'moral', 'guion', 'rey', 'pescador', 'guion', 'frenet', 'relacion', 'familiar', 'dificil', 'personaj', 'buen', 'imagin', 'ardnt', 'hor', 'vid', 'interes', 'coherent', 'furgonet', 'viaj', 'larg', 'teni', 'material', 'aburr', 'crec', 'leccion', 'peli', 'divert', 'ocasion', 'cos', 'olvid', 'perdon', 'kinn', 'mejor', 'papel', 'carrell', 'enorm', 'papel', 'suic', 'loc', 'arkin', 'tem', 'leccion', 'genial', 'papel', 'contrari', 'dificil', 'papel', 'millon', 'vec', 'mejor', 'atras', 'mental', 'jennif', 'dre

[1.000e+00 8.044e+03 5.218e+03]
['aplaus', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'plas', 'pelicul', 'mund', 'imaginari', 'guion', 'padr', 'abuel', 'cas', 'par', 'vertical', 'willy', 'wonk', 'due', 'mayor', 'imperi', 'chocolater', 'fabric', 'mejor', 'chocolatin', 'fabric', 'ningun', 'trabaj', 'misteri', 'cartel', 'fabric', 'billet', 'dor', 'interior', 'chocolatin', 'wonk', 'ores', 'itos', 'rios', 'chocolat', 'palaci', 'caca', 'ascensor', 'magic', 'suces', 'increibl', 'personaj', 'impos', 'cosit', 'burton', 'hech', 'plagi', 'cog', 'puest', 'palabr', 'rar', 'pelicul', 'encant', 'reid', 'emocion', 'sufr

['fin', 'tach', 'babel', 'list', 'hor', 'encant', 'mejor', 'pelicul', 'vist', 'prim', 'largometraj', 'alejandr', 'arritu', 'amor', 'perr', 'gust', 'muchisim', 'gram', 'dej', 'fri', 'babel', 'entend', 'larg', 'pas', 'histori', 'enganch', 'final', 'tragic', 'deslig', 'rest', 'maner', 'dram', 'evident', 'interpret', 'actor', 'espectacular', 'agil', 'human', 'director', 'puest', 'escen', 'magnif', 'dificil', 'guion', 'guillerm', 'arriag', 'consegu', 'coherent', 'superficial', 'mund', 'diferent', 'tragedi', 'sentimental', 'ternur', 'imperfeccion', 'error', 'personaj', 'dobl', 'habri', 'mejor', 'version', 'original', 'cas', 'babel', 'consigui', 'vez', 'dificil', 'med', 'sali', 'cin', 'emocion', 'toc', 'diferent', 'habi', 'entrad', 'dejeis']
[1.000e+00 7.880e+03 3.991e+03]
['dad', 'panoram', 'cinematograf', 'actual', 'seri', 'descabell', 'creacion', 'figur', 'defensor', 'espect', 'gigant', 'molin', 'vient', 'solitari', 'salvaguard', 'honr', 'dignid', 'public', 'apoy', 'podri', 'consumidor', '

In [25]:
X


array([[1.0000e+00, 4.9460e+03, 3.4480e+03],
       [1.0000e+00, 1.2191e+04, 6.7100e+03],
       [1.0000e+00, 2.8681e+04, 1.4213e+04],
       ...,
       [1.0000e+00, 5.7810e+03, 3.8590e+03],
       [1.0000e+00, 1.0192e+04, 5.4420e+03],
       [1.0000e+00, 8.8460e+03, 5.3750e+03]])

In [26]:
X.shape

(609, 3)

In [27]:
Y = y_train

Luego de aplicar la extraccion de caracteŕisticas, aplicamos el algoritmo de gradiente descendente en la funcíon logística.

In [28]:
J, theta = gradientDescent(X, Y, np.zeros((3, 1)), 1e-9, 100000)
print(f"The cost after training is {J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(theta)]}")

The cost after training is 0.27456547.
The resulting vector of weights is [-2.2e-06, 0.00230216, -0.00404168]


### El vector de pesos resultante, se usa para predecir y testear la precision del algoritmo.


In [29]:

def predict_tweet(tweet, freqs, theta):
    '''
    Input: 
        tweet: a string
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
        theta: (3,1) vector of weights
    Output: 
        y_pred: the probability of a tweet being positive or negative
    '''
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    
    # extract the features of the tweet and store it into x
    x = extract_features(tweet,freqs)
    
    # make the prediction using x and theta
    y_pred = sigmoid(np.dot(x,theta))
    
    ### END CODE HERE ###
    
    return y_pred

In [30]:

print( '%s -> %f real: %f' % (x_test[26,0], predict_tweet(x_test[26,0], freqs, theta), y_test[26,0]))

['director', 'escas', 'filmografi', 'espald', 'arrop', 'pes', 'pes', 'industri', 'maner', 'primer', 'trabaj', 'actor', 'prestigi', 'sam', 'nelly', 'demostr', 'talent', 'escas', 'pelicul', 'telefilm', 'autent', 'produccion', 'cinematograf', 'irresist', 'subproduct', 'talent', 'protagon', 'mediocr', 'unic', 'irresist', 'pelicul', 'gan', 'irte', 'sal', 'tiemp', 'dram', 'tint', 'suspens', 'rev', 'sabri', 'argument', 'torn', 'muj', 'trabaj', 'mar', 'cas', 'coch', 'espos', 'creenci', 'generaliz', 'paran', 'eje', 'central', 'temp', 'cansin', 'monoton', 'unid', 'guion', 'practic', 'plan', 'irresist', 'pelicul', 'vulg', 'buen', 'segur', 'pen', 'glori', 'carteler', 'hor', 'despu', 'mencion', 'sarandon', 'desgraci', 'grand', 'estudi', 'produccion', 'menor', 'quiz', 'siti', 'adecu', 'irresist', 'estanteri', 'vide', 'club', 'arrincon', 'alquil', 'libr', 'diner'] -> 0.195974 real: 0.000000


In [31]:
test_logistic_regression(x_test, y_test, freqs, theta)

0.8374384236453202

Como resultado tenemos una precisión de 79.8% dadas por falsos positivos y positivos falsos, lo cual el algoritmo va a fallar en aproximadamente en 1 de 5 reviews que se le carguen. Hay que tener en cuenta que se usó un dataset pequeño,ademas de que es necesario mejorar el proceso de limpioza, pero con esto se muestra que es posible usar machine learning para determinar si una pelicula es buena o mala. Finalmente guaradeamos el valor de los pesos para usarlo en otras predicciones.

In [32]:
import pickle
outfile = open('Data/recsys','wb')
info = {"freqs":freqs,
       "theta":theta}
pickle.dump(info,outfile)
outfile.close()
